In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


import libraries

In [ ]:
import pandas as pd
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import re
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
from sklearn.linear_model import PassiveAggressiveClassifier

downloads

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

Load data

In [ ]:
df=pd.read_csv('/content/drive/MyDrive/datascience_file/fake-news.csv')

Explore data

In [ ]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [ ]:
# check for null values
df.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [ ]:
#copy of df
df_copy = df.copy()

In [ ]:
# drop null values
df_copy.dropna(inplace=True)

In [ ]:
#reset the index for easy iteration
df_copy.reset_index(inplace=True)

In [ ]:
#check for length of data
len(df_copy)

18285

In [ ]:
#check for no. rows and columns
df_copy.shape

(18285, 6)

In [ ]:
#check for unique labels
print(df_copy.label.unique())

[1 0]


divide the data into independent(x) and dependent variables(y)

In [ ]:
x=df_copy.title
y=df_copy.label

In [ ]:
print(len(x))

18285


In [ ]:
stm = PorterStemmer()
corpus = []
for i in range(18285):
    msg = re.sub('[^a-zA-Z]', ' ',x[i])
    msg = msg.lower()
    msg = msg.split()
    
    msg = [stm.stem(word) for word in msg if not word in stopwords.words('english')]
    msg = ' '.join(msg)
    corpus.append(msg)

Convert corpus into doc matrix using tf-idf

In [ ]:
## TFidf Vectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_v=TfidfVectorizer(max_features=5000,ngram_range=(1,3))
x=tfidf_v.fit_transform(corpus).toarray()

In [ ]:
#get fetures names
tfidf_v.get_feature_names()[:6]

['abandon', 'abc', 'abc news', 'abduct', 'abe', 'abedin']

Split into train and test

In [ ]:
## Divide the dataset into train and test set
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state=0)

model buiding

Random Forest Classifier

In [ ]:
# implement RandomForest Classifier
randomclassifier=RandomForestClassifier(n_estimators=200,criterion='entropy').fit(x_train,y_train)

In [ ]:
y_pred = randomclassifier.predict(x_test)

In [ ]:
#lets check the performance of the random forest model
matrix=confusion_matrix(y_test,y_pred)
print(matrix)
score=accuracy_score(y_test,y_pred)
print(score)
report=classification_report(y_test,y_pred)
print(report)

[[3057  332]
 [  85 2561]]
0.9309030654515327
              precision    recall  f1-score   support

           0       0.97      0.90      0.94      3389
           1       0.89      0.97      0.92      2646

    accuracy                           0.93      6035
   macro avg       0.93      0.93      0.93      6035
weighted avg       0.93      0.93      0.93      6035



Passive Aggressive Classifier Algorithm

In [ ]:
#initialise the model
clf = PassiveAggressiveClassifier(max_iter=55)

In [ ]:
#fit the data
clf.fit(x_train, y_train)

PassiveAggressiveClassifier(C=1.0, average=False, class_weight=None,
                            early_stopping=False, fit_intercept=True,
                            loss='hinge', max_iter=55, n_iter_no_change=5,
                            n_jobs=None, random_state=None, shuffle=True,
                            tol=0.001, validation_fraction=0.1, verbose=0,
                            warm_start=False)

In [ ]:
#predict the test data
y_pred = clf.predict(x_test)


In [ ]:
#evaluate the model
print("accuracy:   %0.2f" % accuracy_score(y_test, y_pred))
print('confusion matrix:',confusion_matrix(y_test, y_pred))


accuracy:   0.92
confusion matrix: [[3090  299]
 [ 182 2464]]
